# Finesse 3 - Simple Michelson Validation Test

Test that Finesse 3 correctly reproduces the response of a simple Michelson interferometer. Defining the distance along to two arms of the interferometer to be $L_x$ and $L_y$, then the common and differential arm lengths are,
\begin{align}
\overline{L} &= \frac{L_x+L_y}{2},\\
\Delta L &= L_x-L_y.
\end{align}

![Drawing of the simple Michelson](simple_michelson_drawing.png)

We know from equation 5.11 in [1] that the electric field at the antisymetric port for such a michelson is
\begin{equation}
E_S = E_0 i e^{2ik \overline{L}} cos(k\Delta L)
\end{equation}

This simple Michelson is defined to have $1\,$W of input laser power, $1\,$m arms and a 50:50 beamsplitter in the following Kat code. The tuning paraameter may then be used to adjust the $x$ mirror position in units of $kL$ degrees.

[1] Bond, C., Brown, D., Freise, A. et al. Living Rev Relativ (2016) 19: 3. https://doi.org/10.1007/s41114-016-0002-8

In [1]:
import sys, os
import finesse
import numpy as np

kat = finesse.Model()
kat.parse("""
l laser P=1
s si laser.p1 BS.p1 L=1
bs BS R=0.5 T=0.5
s LX BS.p3 mX.p1 L=1
m mX R=1 T=0
s LY BS.p2 mY.p1 L=1
m mY R=1 T=0
ad pout BS.p4.o 0
xaxis mX.phi lin (-90) 90 200
""")

out = kat.run()
tuning = np.pi*out.x1/180

     finesse.script.lexer [ WARNING]: Name 'laser' is also the name of a directive, which may lead to confusion.


The power response of a michelson is 
\begin{equation}
P = P_0 cos^2(k\Delta L).
\end{equation}

In [2]:
power_theory = np.power(np.cos(tuning),2)
power_finesse = (out['pout']*np.conj(out['pout']))

# The power should be real
try:
    assert np.all(power_finesse.imag == 0)
except AssertionError:
    print('Whilst calculating the power from field'
          ' amplitude an imaginary power was detected!')
    print('Power calculated by taking amp*np.conj(amp)')
    print('Power = {}'.format(pd))
    sys.exit(15)
else:
    print('Power is real.')

# Compare analytics and Finesse
maxerr = np.max(power_theory-power_finesse.real)
print('Maximum Power Error: {:.1e}W'.format(maxerr))
print('Max relative difference: '+str(maxerr))
try:
    assert np.allclose(power_finesse,power_theory,
                       atol=3e-14,rtol=3e-14)
except AssertionError:
    print('Power returned from simple michson was not '
          'accurate to within 3*10^{-14}.')
    sys.exit(15)
else:
    print('Power values match analytic.')

Power is real.
Maximum Power Error: 1.4e-16W
Max relative difference: 1.3877787807814457e-16
Power values match analytic.


Finesse positions optics at $\lambda \text{floor}(L/\lambda)$ for distance $L$ to mitigate floating-point errors. Accounting for this, the electric field at the output photodiode may be computed and the numeric and analytic results compared.

In [3]:
#< ---- 20 char ---->< ---- 20 char ---->< ---- 20 char ----><9 char->
Lambda = 1064e-9
k = 2*np.pi/Lambda

# Common arm length (in units of angular frequency)
common_arm = 0.5*(k*(Lambda*np.floor((1+1)/Lambda))+tuning)

# Differential arm length (in units of angular frequency)
differential_arm = tuning

# Electric field at photodiode (analytics)
E_theory = (1j)*np.exp(2j*common_arm)*np.cos(differential_arm)

# Electric field at photodide (Finesse)
E_finesse = out['pout']

In [4]:
amplitude_err = np.abs(E_finesse)-np.abs(E_theory)
phase_err = np.angle(E_theory)-np.angle(E_finesse)

print('Maximum Amplitude Error: {:.1e} sqrt(W)'.format(
    np.max(np.abs(amplitude_err))))
print('Maximum Phase Error: {:.1e} radians'.format(
    np.max(np.abs(phase_err))))

try:
    assert np.allclose(amplitude_err,0,atol=1e-15,rtol=1e-15)
except AssertionError:
    print('Electric field amplitude returned from simple '
          'Michelson was not accurate to within 3*10^{-15}.')
    sys.exit(15)
else:
    print('Amplitude matches analytics')
    
try:
    assert np.allclose(phase_err,0,atol=1e-8,rtol=1e-15)
except AssertionError:
    print('Electric field phase returned from simple Michelson'
          ' was not accurate to within 3*10^{-8}.')
    sys.exit(15)
else:
    print('Phase matches analytics')

Maximum Amplitude Error: 5.6e-16 sqrt(W)
Maximum Phase Error: 2.1e-09 radians
Amplitude matches analytics
Phase matches analytics


In [5]:
if 'BESTDATASERVER' in os.environ:
    from bestutils import do_upload
else:
    def do_upload(*args,**kwargs): pass
    
from matplotlib import pyplot as plt
fig, ax = plt.subplots(2,2,sharex=True)
ax[0,0].plot(tuning,np.abs(E_theory),'b-',label='Theory')
ax[0,0].plot(tuning,np.abs(E_finesse),'r--',label='Finesse')
ax[0,0].set_ylabel('Amplitude [$\sqrt{\mathrm{W}}$]')

ax[0,1].plot(tuning,np.angle(E_theory),'b-')
ax[0,1].plot(tuning,np.angle(E_finesse),'r--')
ax[0,1].set_ylabel('Phase [rad]')

ax[1,0].plot(tuning,amplitude_err,'k')
ax[1,0].set_ylabel(r'Amplitude Error [$\sqrt{\mathrm{W}}$]')
ax[1,0].set_xlabel('Tuning [rad]')

ax[1,1].plot(tuning,phase_err,'k')
ax[1,1].set_ylabel('Phase Error [rad]')
ax[1,1].set_xlabel('Tuning [rad]')
for a in ax.flat:
    a.grid()

fig.legend()
fig.suptitle('Simple Michelson Error Behaviour')
#fig.tight_layout()
file = 'simple_michelson_output.png'
fig.savefig(file)
plt.close()

do_upload(file)

Error behaviour for Finesse3 simple Michelson
![Drawing of the Simple Michelson](simple_michelson_output.png)